# RAG 多檔案與元數據

本範例展示：
1. **第1個儲存格**：建立多檔案向量資料庫（含元數據）
2. **第2個儲存格**：使用元數據過濾查詢

學習目標：理解如何處理多個來源的文件並使用元數據進行過濾檢索

In [1]:
# 第1個儲存格：建立多檔案向量資料庫（含元數據）

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
books_dir = os.path.join(current_dir, "books")
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

print(f"書籍目錄: {books_dir}")
print(f"持久化目錄: {persistent_directory}")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 確保書籍目錄存在
    if not os.path.exists(books_dir):
        raise FileNotFoundError(
            f"目錄 {books_dir} 不存在。請檢查路徑。"
        )

    # 列出目錄中所有文字檔案
    book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

    # 從每個檔案讀取文字內容並儲存元數據
    documents = []
    for book_file in book_files:
        file_path = os.path.join(books_dir, book_file)
        loader = TextLoader(file_path)
        book_docs = loader.load()
        for doc in book_docs:
            # 為每個文件添加元數據以指示其來源
            doc.metadata = {"source": book_file}
            documents.append(doc)

    # 將文件分割成塊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立並持久化向量存儲
    print("\n--- 正在建立並持久化向量存儲 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- 完成建立並持久化向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

書籍目錄: /Users/roberthsu2003/Documents/GitHub/langchain_project/4_rag/books
持久化目錄: /Users/roberthsu2003/Documents/GitHub/langchain_project/4_rag/db/chroma_db_with_metadata_chinese_nb
持久化目錄不存在。正在初始化向量存儲...

--- 文件塊資訊 ---
文件塊數量: 77

--- 正在建立嵌入 ---


/var/folders/7g/kb_wcfhj3710_xnyqdgx_v940000gn/T/ipykernel_3819/1965316693.py:52: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermedia


--- 完成建立嵌入 ---

--- 正在建立並持久化向量存儲 ---

--- 完成建立並持久化向量存儲 ---


In [2]:
# 第2個儲存格：使用元數據過濾查詢

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "路由器的WiFi密碼如何更改？"

# 根據查詢檢索相關文件
# search_type="similarity_score_threshold": 使用相似度分數閾值過濾
# k=3: 返回最多 3 個相關文件
# score_threshold=0.1: 較低的閾值，可以返回更多相關文件
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")

# 進階查詢：只從特定來源檢索
print("\n--- 只從《路由器設定手冊》檢索 ---")
# 使用 filter 參數指定元數據條件
specific_docs = db.similarity_search(
    query,
    k=3,
    filter={"source": "路由器設定手冊.txt"}
)

for i, doc in enumerate(specific_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi


--- 相關文件 ---
文件 1:
1. 消費者保護:
   - 消費者服務專線:1950
   - 各縣市消費者服務中心

2. 租賃相關:
   - 內政部不動產資訊平台
   - 各縣市政府地政局

三、司法途徑

1. 支付命令:
   - 適用:租金、押金返還等金錢請求
   - 向法院聲請
   - 快速但對方可異議

2. 民事訴訟:
   - 小額訴訟:10萬元以下
   - 簡易訴訟:50萬元以下
   - 通常訴訟:50萬元以上

3. 強制執行:
   - 取得執行名義(判決、和解等)
   - 向法院聲請強制執行

四、法律扶助

法律扶助基金會:
- 服務專線:412-8518(手機加02)
- 提供法律諮詢、訴訟扶助
- 符合資格者免費

第九部分: 實用檢查表

一、看屋檢查表

□ 房屋基本資訊確認
  □ 地址、樓層、坪數
  □ 格局、座向
  □ 屋齡、建物型態

□ 房屋狀況檢查
  □ 牆壁、天花板無漏水痕跡
  □ 地板平整無龜裂
  □ 門窗開關正常
  □ 採光、通風良好
  □ 無壁癌、發霉

□ 水電設施檢查
  □ 水龍頭出水正常
  □ 馬桶沖水正常
  □ 排水順暢
  □ 插座數量足夠且正常
  □ 電燈開關正常
  □ 熱水器運作正常

□ 設備清點
  □ 冷氣運轉正常
  □ 家具家電清單
  □ 設備新舊狀況

□ 安全檢查
  □ 消防設備(滅火器、偵煙器)
  □ 逃生路線
  □ 對講機、門鎖

□ 周邊環境
  □ 交通便利性
  □ 生活機能
  □ 治安狀況
  □ 噪音狀況

二、簽約檢查表

□ 契約內容確認
  □ 雙方基本資料正確
  □ 房屋地址、範圍明確
  □ 租金金額、繳納方式
  □ 押金金額
  □ 租賃期間
  □ 修繕責任劃分
  □ 提前終止條件
  □ 費用負擔約定

□ 特約事項
  □ 設備清單
  □ 使用限制
  □ 其他約定

□ 附件文件
  □ 房屋所有權狀影本
  □ 屋況、設備照片
  □ 水電表度數記錄

□ 簽約程序
  □ 逐條閱讀理解
  □ 不明處詢問清楚
  □ 雙方簽名蓋章
  □ 各執一份契約

三、入住檢查表

□ 房屋狀況記錄
  □ 拍照記錄現況
  □ 標註原有損壞
  □ 記錄水電表度數

來源: 租屋契約範本與

/var/folders/7g/kb_wcfhj3710_xnyqdgx_v940000gn/T/ipykernel_3819/1117769952.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persistent_directory,


In [3]:
# 第3個儲存格：整合 Chain - 多來源文件的智慧問答鏈

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義使用者的問題
question = "如何保養洗衣機和路由器？"

print(f"❓ 使用者問題: {question}\n")

# 步驟 1: 從多個來源檢索相關文件
print("=" * 70)
print("步驟 1: 從多個來源檢索相關文件")
print("=" * 70)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 取4個相關文件（可能來自不同手冊）
)

retrieved_docs = retriever.invoke(question)

print(f"✅ 找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    source = doc.metadata.get('source', 'Unknown')
    print(f"📄 文件 {i} - 來源: {source}")
    print(f"   內容: {doc.page_content[:80]}...\n")

# 步驟 2: 建立帶有來源標註的 RAG Chain
print("=" * 70)
print("步驟 2: 建立多來源 RAG Chain")
print("=" * 70)

# 定義提示模板（強調來源資訊）
template = """你是一個專業的產品客服助手。請根據以下多個產品手冊的參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

回答要求：
1. 針對不同產品分別說明（如果問題涉及多個產品）
2. 在回答中標註資訊來源（例如：「根據洗衣機使用說明...」）
3. 只根據參考資料回答，不要編造內容
4. 如果某個產品的資訊不足，請說明
5. 用條列式整理，讓使用者容易閱讀

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 建立 LLM
llm = ChatOllama(model="llama3.2", temperature=0)

# 定義文件格式化函數（包含來源資訊）
def format_docs_with_source(docs):
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"【來源 {i}: {source}】\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# 建立完整的 RAG Chain（整合 3_chains 的技巧）
rag_chain = (
    {
        "context": retriever | format_docs_with_source, 
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ 多來源 RAG Chain 建立完成\n")

# 步驟 3: 執行 Chain 並取得整合答案
print("=" * 70)
print("步驟 3: 執行 Chain 取得整合答案")
print("=" * 70)

answer = rag_chain.invoke(question)

print(f"\n🤖 【AI 整合回答】\n")
print(answer)
print()

# 步驟 4: 示範使用元數據過濾的 Chain
print("=" * 70)
print("步驟 4: 只查詢特定來源的 Chain")
print("=" * 70)

specific_question = "路由器如何設定？"
print(f"❓ 問題: {specific_question}\n")

# 創建只查詢路由器手冊的檢索器
def retrieve_from_router_manual(query):
    docs = db.similarity_search(
        query,
        k=3,
        filter={"source": "路由器設定手冊.txt"}
    )
    return docs

# 建立特定來源的 Chain
specific_chain = (
    {
        "context": RunnablePassthrough() | retrieve_from_router_manual | format_docs_with_source,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

specific_answer = specific_chain.invoke(specific_question)
print(f"🤖 【只查路由器手冊的回答】\n")
print(specific_answer)
print()

print("=" * 70)
print("📚 多來源 RAG Chain 的優勢")
print("=" * 70)
print("✅ 可以同時查詢多個產品手冊")
print("✅ 自動標註資訊來源，方便追蹤")
print("✅ 可以用 metadata 過濾特定來源")
print("✅ 整合多個文件的資訊給出完整答案")
print("\n💡 這展示了 Chain 3 中學到的組合技巧在 RAG 中的應用！")

❓ 使用者問題: 如何保養洗衣機和路由器？

步驟 1: 從多個來源檢索相關文件
✅ 找到 4 個相關文件

📄 文件 1 - 來源: 租屋契約範本與說明.txt
   內容: 1. 消費者保護:
   - 消費者服務專線:1950
   - 各縣市消費者服務中心

2. 租賃相關:
   - 內政部不動產資訊平台
   - 各縣市政府...

📄 文件 2 - 來源: 路由器設定手冊.txt
   內容: 2.3 使用手機 App設定
----------------------------------------
步驟 1：下載 App
- iOS：至 App ...

📄 文件 3 - 來源: 洗衣機使用說明.txt
   內容: 問題:衣物有洗劑殘留
可能原因與解決方法:
1. 洗劑用量過多
2. 選擇額外漂洗功能
3. 水溫過低導致洗劑溶解不完全
4. 使用低泡沫洗劑

6.2 錯誤代...

📄 文件 4 - 來源: 路由器設定手冊.txt
   內容: 線上支援：
官方網站：www.taiwanrouter.com.tw
電子信箱：support@taiwanrouter.com.tw
Line官方帳號：@ta...

步驟 2: 建立多來源 RAG Chain
✅ 多來源 RAG Chain 建立完成

步驟 3: 執行 Chain 取得整合答案

🤖 【AI 整合回答】

以下是針對不同產品分別的保養指導：

**洗衣機**

根據洗衣機使用說明（第八章），以下是保養的建議：

* 清潔前務必拔掉電源插頭
* 請勿用水直接沖洗洗衣機
* 請勿使用強酸強鹼清潔劑
* 維修時請聯絡專業人員

**路由器**

根據路由器設定手冊（第九部分），以下是保養的建議：

* 定期更新路由器的軟體和 firmware
* 關閉不需要使用的連線
* 使用適當的清潔劑清潔路由器外殼
* 避免使用延長線或多孔插座

如果您有任何其他問題或疑問，請聯絡我們的客服專線：0800-XXX-XXX

步驟 4: 只查詢特定來源的 Chain
❓ 問題: 路由器如何設定？

🤖 【只查路由器手冊的回答】

1.  **針對不同產品分別說明**：根據參考資料，我們可以看到路由器設定手冊中提到了三個不同的產品：TR Router、TR-5000_XXXX等。每個產品都有其 own 